In [5]:
%pip install scikit-learn
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 334.5 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.3.0-py3-none-macosx_14_0_arm64.whl size=1408565 sha256=a5d08805cbafabf90d0d5eedea377b09830152e9065f7c3f25ad9a16a8f0ac11
  Stored in directory: /Users/sahilahuja/Library/Caches/pip/wheels/f4/d1/f4/3cd66516e5a05b0879846879dfb06e5a087f453adb405a5999
Successfully built lightgbm
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

sample = pd.read_csv('../data/raw/sample_submission.csv')
df_test = pd.read_csv('../data/raw/test.csv')
df_train = pd.read_csv('../data/raw/train.csv')

df_train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feature_cols = [col for col in df_train.columns if col.startswith('var_')]
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])

In [7]:
df_train.head(1)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.53643,-0.334926,0.608751,...,0.263374,-1.149158,0.817469,-0.411013,0.168705,-1.578117,1.022131,-0.373968,-1.026398,0.214135


In [13]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Define LightGBM parameters
lgb_params = {
    "objective": "binary",
    "metric": "auc",
    "boosting": 'gbdt',
    "max_depth": -1,
    "num_leaves": 31,
    "learning_rate": 0.01,
    "bagging_freq": 5,
    "bagging_fraction": 0.2,
    "feature_fraction": 0.1,
    "min_data_in_leaf": 100,
    "tree_learner": "data",
    "boost_from_average": "false",
    "verbosity": 2
}

# Split data
X = df_train.drop(['ID_code', 'target'], axis=1)
y = df_train['target']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=33)

# Train model
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)
lgb_model = lgb.train(lgb_params, train_data, valid_sets=[val_data], num_boost_round=10000, 
                      callbacks = [
                          lgb.early_stopping(stopping_rounds=200)
                      ])

[LightGBM] [Info] Number of positive: 12153, number of negative: 107847
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000008 seconds, init for row-wise cost 0.029807 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030690 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51000
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 200
[LightGBM] [Debug] Re-bagging, using 24041 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
Training until validation scores don't improve for 200 rounds
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [

In [18]:
import joblib

joblib.dump(lgb_model, 'lgb_clf.pkl')
gbm_pickle = joblib.load('lgb_clf.pkl')

In [22]:
X_train

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
115253,0.762188,0.914294,1.385674,-1.612346,1.625897,1.325943,-0.855117,0.600383,-0.306294,-0.141398,...,-1.007748,-0.483421,0.248483,0.839131,0.840602,-0.431807,0.576065,-0.528261,-0.922742,0.340960
12271,-0.307270,0.190695,0.194483,-0.676905,-2.309240,0.855946,-0.837462,-0.625046,0.140381,-0.964188,...,-0.212864,0.306884,-1.003598,-1.907799,0.488721,1.640366,1.806660,-0.099995,1.543931,0.378496
153887,-1.013609,-1.107442,0.531453,1.361499,-0.782698,1.211563,-0.095717,-0.627474,1.826740,0.215910,...,-0.148981,0.402741,0.758081,-0.503798,1.414609,-0.383534,-0.381811,0.028691,0.449554,0.641880
31981,1.165735,-0.284165,-0.062590,0.276644,0.821409,-0.748251,-0.375891,0.598540,-1.092941,1.744815,...,-0.238588,-0.146963,1.021200,0.747297,-1.466494,0.472088,1.153420,1.116446,-2.613544,-1.520347
31050,0.257590,1.741201,1.107093,0.189775,0.144267,2.263355,0.585101,0.766852,-1.254435,-0.631088,...,-1.722740,-0.533598,-0.530119,1.122095,1.168624,0.133337,0.095550,-0.635138,-1.664438,-1.725080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158921,-1.287782,-0.334683,-0.489984,1.222216,-1.394843,0.821533,-0.360313,0.050072,0.358948,-0.083102,...,0.202758,0.285781,-0.834904,-0.078174,-0.909998,0.207426,0.124811,0.369286,-0.174105,2.106549
131650,0.694624,1.070786,0.591130,0.470496,-1.311610,0.952370,-0.552326,-1.625085,0.362278,-1.971421,...,-0.059834,1.074763,1.156209,-1.416519,0.715919,1.427615,-0.466808,-0.256784,2.006976,-0.410171
104152,0.276208,-0.547768,-0.880193,-1.617142,-0.427216,-0.975918,-0.330310,1.803022,0.242643,-1.070660,...,1.338987,-0.686380,0.269383,0.816535,0.178433,0.674136,-0.264896,1.435557,0.785662,0.074175
199047,-1.455543,-0.502410,2.276321,2.316419,0.103236,-0.328755,-0.556826,0.832942,0.326421,0.819117,...,0.673974,-2.139809,-0.232639,-0.948310,-0.632276,-1.431339,-0.215302,-1.154113,-0.627386,-0.743109
